In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydtmc import MarkovChain

In [2]:
team_index = ["FKB Power Cats Düdingen", "Genève Volley", "Raiffeisen Volley Glaronia", "Raiffeisen Volley Toggenburg", "Sm'Aesch Pfeffingen", "VBC Cheseaux", "VC Kanti Schaffhausen", "Viteos NUC", "Volley Lugano", "Volleyball Academy"]
df = pd.read_csv("stats_nla_women.csv")
df['Date'] = pd.to_datetime(df["Date"], format='%d.%m.%Y')

df_results = pd.read_csv("results_nla_women.csv")
df_results['Date'] = pd.to_datetime(df_results["Date"], format='%d.%m.%Y')
playoff_start = "2024-03-02"
df_playoff = df_results[df_results["Date"] >= playoff_start]

for idx in df.index:
    
    if (idx % 2 == 0):
        df.loc[idx + 1, 'OpponentAttack'] = df.loc[idx, 'AttackTot']
    else:
        df.loc[idx - 1, 'OpponentAttack'] = df.loc[idx, 'AttackTot']


In [3]:
team_percentages = pd.DataFrame(columns=['ServeContinue','ServeAce','ServeError',
                                         'ReceptionPositive','ReceptionNegativ','ReceptionError',
                                         'Block',
                                         'PosAttackError','PosAttackPoint','PosAttackContinue',
                                         'NegAttackError','NegAttackPoint','NegAttackContinue',
                                         'DigAttackError','DigAttackPoint','DigAttackContinue',], index=team_index)

for team in team_index:
    team_df = df[df['Team'] == team]

    team_percentages.at[team,'ServeAce'] = team_df['ServePts'].sum() / team_df['ServeTot'].sum()
    team_percentages.at[team,'ServeError'] = team_df['ServeErr'].sum() / team_df['ServeTot'].sum()
    team_percentages.at[team,'ServeContinue'] = 1 - team_percentages.at[team,'ServeAce'] - team_percentages.at[team,'ServeError']

    team_percentages.at[team,'ReceptionError'] = team_df['RecErr'].sum() / team_df['RecTot'].sum()

    team_percentages.at[team,'Block'] = team_df['Block'].sum() / team_df['OpponentAttack'].sum()

df = df.dropna()

for idx in df.index:
    df.loc[idx, 'AfterPosPts'] = round(0.01 * df.loc[idx, 'AfterPosPts'] * df.loc[idx, 'AfterPosTot'])
    df.loc[idx, 'AfterNegPts'] = round(0.01 * df.loc[idx, 'AfterNegPts'] * df.loc[idx, 'AfterNegTot'])
    df.loc[idx, 'AfterDigPts'] = round(0.01 * df.loc[idx, 'AfterDigPts'] * df.loc[idx, 'AfterDigTot'])


for team in team_index:
    team_df = df[df['Team'] == team]

    team_percentages.at[team,'ReceptionPositive'] = team_df['AfterPosTot'].sum() / (team_df['RecTot'].sum())
    team_percentages.at[team,'ReceptionNegativ'] = 1 - team_percentages.at[team,'ReceptionPositive'] - team_percentages.at[team,'ReceptionError']

    team_percentages.at[team,'PosAttackError'] = team_df['AfterPosErr'].sum() / (team_df['AfterPosTot'].sum())
    team_percentages.at[team,'PosAttackPoint'] = team_df['AfterPosPts'].sum() / (team_df['AfterPosTot'].sum())
    team_percentages.at[team,'PosAttackContinue'] = 1 - team_percentages.at[team,'PosAttackError'] - team_percentages.at[team,'PosAttackPoint']

    team_percentages.at[team,'NegAttackError'] = team_df['AfterNegErr'].sum() / (team_df['AfterNegTot'].sum())
    team_percentages.at[team,'NegAttackPoint'] = team_df['AfterNegPts'].sum() / (team_df['AfterNegTot'].sum())
    team_percentages.at[team,'NegAttackContinue'] = 1 - team_percentages.at[team,'NegAttackError'] - team_percentages.at[team,'NegAttackPoint']

    team_percentages.at[team,'DigAttackError'] = team_df['AfterDigErr'].sum() / (team_df['AfterDigTot'].sum())
    team_percentages.at[team,'DigAttackPoint'] = team_df['AfterDigPts'].sum() / (team_df['AfterDigTot'].sum())
    team_percentages.at[team,'DigAttackContinue'] = 1 - team_percentages.at[team,'DigAttackError'] - team_percentages.at[team,'DigAttackPoint']

team_percentages

,ServeContinue,ServeAce,ServeError,ReceptionPositive,ReceptionNegativ,ReceptionError,Block,PosAttackError,PosAttackPoint,PosAttackContinue,NegAttackError,NegAttackPoint,NegAttackContinue,DigAttackError,DigAttackPoint,DigAttackContinue
FKB Power Cats Düdingen,0.842491,0.065324,0.092186,0.535552,0.390318,0.07413,0.074074,0.05226,0.378531,0.569209,0.07381,0.297619,0.628571,0.060472,0.34882,0.590708
Genève Volley,0.796466,0.080497,0.123037,0.4375,0.487952,0.074548,0.066211,0.120482,0.428571,0.450947,0.078895,0.311637,0.609467,0.082537,0.326672,0.590791
Raiffeisen Volley Glaronia,0.822819,0.065101,0.112081,0.483796,0.427267,0.088937,0.046551,0.08453,0.422648,0.492823,0.10084,0.264706,0.634454,0.070285,0.303381,0.626335
Raiffeisen Volley Toggenburg,0.847357,0.069248,0.083395,0.444574,0.474262,0.081164,0.042209,0.070866,0.375328,0.553806,0.064706,0.258824,0.676471,0.077836,0.275726,0.646438
Sm'Aesch Pfeffingen,0.810865,0.067069,0.122066,0.46744,0.448031,0.08453,0.062763,0.112595,0.496183,0.391221,0.157248,0.321867,0.520885,0.112903,0.370968,0.516129
VBC Cheseaux,0.783724,0.065217,0.151059,0.473004,0.44943,0.077567,0.07256,0.072347,0.483923,0.44373,0.097996,0.336303,0.565702,0.090715,0.389541,0.519744
VC Kanti Schaffhausen,0.811508,0.078704,0.109788,0.440972,0.482167,0.076861,0.090107,0.106299,0.462598,0.431102,0.103118,0.326139,0.570743,0.106204,0.369085,0.524711
Viteos NUC,0.756991,0.081891,0.161119,0.440073,0.467833,0.092094,0.088156,0.074844,0.505198,0.419958,0.108959,0.389831,0.501211,0.107759,0.411638,0.480603
Volley Lugano,0.825799,0.060639,0.113561,0.440607,0.488307,0.071086,0.072122,0.068834,0.439771,0.491396,0.082789,0.381264,0.535948,0.08089,0.373104,0.546006
Volleyball Academy,0.757282,0.078417,0.164302,0.3829,0.526394,0.090706,0.067236,0.112621,0.396117,0.491262,0.130672,0.3049,0.564428,0.112432,0.344865,0.542703


In [4]:
p = [[0.2, 0.7, 0.0, 0.1], [0.0, 0.6, 0.3, 0.1], [0.0, 0.0, 1.0, 0.0], [0.5, 0.0, 0.5, 0.0]]
mc = MarkovChain(p, ['A', 'B', 'C', 'D'])
mc.simulate(10, initial_state ='A')

['A', 'A', 'D', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']

In [5]:
def getPointStates():
    teams = [1,2]
    actions = ["S", "P", "B_pos", "B_neg", "B_dig", "A_pos", "A_neg", "A_dig", "Point"]
    states = [f"{action}{team}" for team in teams for action in actions]
    return states

states = getPointStates()
print(states)

['S1', 'P1', 'B_pos1', 'B_neg1', 'B_dig1', 'A_pos1', 'A_neg1', 'A_dig1', 'Point1', 'S2', 'P2', 'B_pos2', 'B_neg2', 'B_dig2', 'A_pos2', 'A_neg2', 'A_dig2', 'Point2']


In [6]:
df_playoff

,Date,TeamA,TeamB,Score,Set1,Set2,Set3,Set4,Set5
90,2024-03-02,FKB Power Cats Düdingen,Genève Volley,3:0,25:12,25:13,25:23,NaN,NaN
91,2024-03-03,Viteos NUC,Raiffeisen Volley Glaronia,3:0,25:16,25:14,25:16,NaN,NaN
92,2024-03-03,Raiffeisen Volley Toggenburg,Volleyball Academy,2:3,25:16,25:15,14:25,20:25,11:15
93,2024-03-03,Sm'Aesch Pfeffingen,VBC Cheseaux,3:0,25:22,25:19,31:29,NaN,NaN
94,2024-03-03,VC Kanti Schaffhausen,Volley Lugano,3:1,25:21,17:25,25:22,25:22,NaN
95,2024-03-09,Raiffeisen Volley Glaronia,Viteos NUC,0:3,20:25,23:25,26:25,NaN,NaN
96,2024-03-09,Genève Volley,FKB Power Cats Düdingen,1:3,25:22,25:27,19:25,22:25,NaN
97,2024-03-09,Volleyball Academy,Raiffeisen Volley Toggenburg,3:0,25:17,25:15,25:23,NaN,NaN
98,2024-03-09,VBC Cheseaux,Sm'Aesch Pfeffingen,3:0,25:22,25:23,25:20,NaN,NaN
99,2024-03-09,Volley Lugano,VC Kanti Schaffhausen,3:2,17:25,25:22,26:28,25:21,18:16


In [7]:
important_index = [91, 93, 90, 94, 92, 106, 105, 102, 103, 119, 118, 113, 115]

In [8]:
simulation_length = 7500

def fill_transition_matrix(trans, row, team):
    i = 0
    if team == 'TeamB':
        i = 9

    trans[0 + i][10 - i] = team_percentages.at[row[team],'ServeContinue']
    trans[0 + i][8 + i] = team_percentages.at[row[team],'ServeAce']
    trans[0 + i][17 - i] = team_percentages.at[row[team],'ServeError']
        
    trans[1 + i][11 - i] = team_percentages.at[row[team],'ReceptionPositive']
    trans[1 + i][12 - i] = team_percentages.at[row[team],'ReceptionNegativ']
    trans[1 + i][17 - i] = team_percentages.at[row[team],'ReceptionError']

    trans[2 + i][14 - i] = 1 - team_percentages.at[row[team],'Block']
    trans[2 + i][8 + i] = team_percentages.at[row[team],'Block']

    trans[3 + i][15 - i] = 1 - team_percentages.at[row[team],'Block']
    trans[3 + i][8 + i] = team_percentages.at[row[team],'Block']

    trans[4 + i][16 - i] = 1 - team_percentages.at[row[team],'Block']
    trans[4 + i][8 + i] = team_percentages.at[row[team],'Block']

    trans[5 + i][4 + i] = team_percentages.at[row[team],'PosAttackContinue']
    trans[5 + i][8 + i] = team_percentages.at[row[team],'PosAttackPoint']
    trans[5 + i][17 - i] = team_percentages.at[row[team],'PosAttackError']
    
    trans[6 + i][4 + i] = team_percentages.at[row[team],'NegAttackContinue']
    trans[6 + i][8 + i] = team_percentages.at[row[team],'NegAttackPoint']
    trans[6 + i][17 - i] = team_percentages.at[row[team],'NegAttackError']

    trans[7 + i][4 + i] = team_percentages.at[row[team],'DigAttackContinue']
    trans[7 + i][8 + i] = team_percentages.at[row[team],'DigAttackPoint']
    trans[7 + i][17 - i] = team_percentages.at[row[team],'DigAttackError']

    trans[8 + i][8 + i] = 1

value = np.zeros((len(states), len(states)))
for index, r in df_playoff.iterrows():
    transitionMatrix = np.zeros((len(states), len(states)))
    if index in important_index:
        fill_transition_matrix(transitionMatrix, r, 'TeamA')
        fill_transition_matrix(transitionMatrix, r, 'TeamB')
        mc = MarkovChain(transitionMatrix, states)

        value = transitionMatrix
        
        point = np.zeros(2)
        for i in range(simulation_length):
            result = mc.simulate(20, initial_state='S1', final_state='Point1', output_indices=False)
            if result[-1] == 'Point1':
                point[0] += 1
            elif result[-1] == 'Point2':
                point[1] += 1

        results_serveB = np.zeros(2)
        for i in range(simulation_length):
            result = mc.simulate(20, initial_state='S2', final_state='Point2', output_indices=False)
            if result[-1] == 'Point1':
                point[0] += 1
            elif result[-1] == 'Point2':
                point[1] += 1
        
        print(r['TeamA'], "vs", r['TeamB'], point)             

FKB Power Cats Düdingen vs Genève Volley [7703. 7243.]
Viteos NUC vs Raiffeisen Volley Glaronia [8039. 6944.]
Raiffeisen Volley Toggenburg vs Volleyball Academy [7384. 7566.]
Sm'Aesch Pfeffingen vs VBC Cheseaux [7210. 7776.]
VC Kanti Schaffhausen vs Volley Lugano [7472. 7511.]
VC Kanti Schaffhausen vs Raiffeisen Volley Glaronia [8022. 6938.]
Genève Volley vs VBC Cheseaux [7331. 7639.]
Sm'Aesch Pfeffingen vs FKB Power Cats Düdingen [7234. 7736.]
Viteos NUC vs Volley Lugano [7622. 7368.]
VC Kanti Schaffhausen vs Genève Volley [7813. 7153.]
VBC Cheseaux vs Raiffeisen Volley Glaronia [8071. 6899.]
Sm'Aesch Pfeffingen vs Volley Lugano [7184. 7800.]
Viteos NUC vs FKB Power Cats Düdingen [7699. 7283.]
